In [119]:
from CirData.Base import CirData
import psycopg2
import pandas as pd

In [95]:
conn = psycopg2.connect("dbname='DICOMDB' user='postgres' host='149.148.109.29' password='npgdb'")
cur = conn.cursor()

In [94]:
SQL_KERNELS = """

WITH series_select AS (SELECT s.*, ROW_NUMBER() OVER(PARTITION BY s.studyuid ORDER BY s.volumeshape[3] DESC, s.dicom_header->'SeriesNumber' ASC) as max_slices from series s JOIN event e ON s.e_id = e.e_id JOIN intersect_proj_event pj ON pj.event_id = e.e_id WHERE
s.flag = 'O' AND
s.modality = 'CT' AND
s.slicingdirection = 'a' AND 
s.dicom_header->'ImageType' ?| array['PRIMARY'] AND
s.dicom_header->'ImageType' ?| array['ORIGINAL'] AND
s.dicom_header->'PatientPosition' ?| array['HFS', 'FFS'] AND
(s.dicom_header->'BodyPartExamined' ?| 
array['Chest', 'CHEST', 'CHEST_ABDOMEN','CHEST-ABDOMEN', 'CT thorax', 'HR_THORAX', 'LUNG', 'Thorax', 'THORAX', 'THORAX ABD', 'THORAX_ABDOMEN', 'THORAX/ABDOMEN','THORAX_HR','THORAX ABDOMEN','Thorax u. Mediastinum','Thx-Abdomen']
OR s.dicom_header->'CodeValue' ?| array['Thorax','RATHLUNG','ctth','cttx','ctthab','CT-HR-Lunge','TH-HR-CT','RATHTHAB','ct-thor','CTTHAB','113691'] OR
LOWER(s.dicom_header->>'SeriesDescription') LIKE '%thorax%' OR 
LOWER(s.dicom_header->>'SeriesDescription') LIKE '%lung%' OR
LOWER(s.dicom_header->>'SeriesDescription') LIKE '%thor%') 

AND (s.dicom_header->>'ConvolutionKernel' LIKE 'B6%')
     
AND CAST(s.dicom_header->'SliceThickness' AS FLOAT) = 1.0
AND s.volumeshape[3] >= 100)

SELECT ss.e_id FROM series_select ss WHERE ss.max_slices = 1

-- INSERT INTO intersect_proj_event (proj_name, event_id) SELECT 'lung_b3_3mm', ss.e_id FROM series_select ss WHERE ss.max_slices = 1 ON CONFLICT DO NOTHING 

"""


# SELECT ss.e_id, ss.volumeshape[3] FROM series_select ss WHERE ss.max_slices = 1 
# SELECT DISTINCT ss.dicom_header->'SeriesDescription', ss.volumeshape[3] FROM series_select ss WHERE ss.max_slices = 1 
# SELECT ss.e_id, ss.volumeshape[3] FROM series_select ss WHERE ss.max_slices = 1 

# studymax AS (SELECT s.studyuid as studyuid, max(s.volumeshape[3]) OVER(PARTITION BY p.customer ORDER BY p.total DESC) AS rk) MAX(s.volumeshape[3]) as maxshape from series_select s GROUP BY s.studyuid),

# SELECT s.studyuid, count(*) from series_select s JOIN studymax ON s.sutdyuid = studymax.studyuid

# WHERE s.volumeshape[3] = studymax.maxshape

# GROUP BY s.studyuid 
# """
# sql_res = cd.query_to_pandas(SQL_KERNELS)
sql_res = cur.execute(SQL_KERNELS)
print(sql_res)
# conn.commit()

SyntaxError: syntax error at or near "ss"
LINE 23: SELECT unique(ss.dicom_header-->'ConvolutionKernel')
                       ^


In [88]:
cd.query_to_pandas(SQL_KERNELS)

,e_id
0,01SBW0BY_20111115_dicom_257560
1,05YCFNCG_20111212_dicom_260822
2,1MCN1JBY_20120423_dicom_207519
3,1FTENQ8K_20120504_dicom_207688
4,05D68JAL_20120813_dicom_210501
...,...
4279,0759FACL_20150303_dicom_253716
4280,000F6RCK_20150316_dicom_252110
4281,03KMJCCG_20150330_dicom_252175
4282,1ELX6K84_20150210_dicom_252202


In [121]:
SQL_KERNELS = """

WITH series_select AS (SELECT s.*, ROW_NUMBER() OVER(PARTITION BY s.studyuid ORDER BY s.volumeshape[3] DESC, s.dicom_header->'SeriesNumber' ASC) as max_slices from series s JOIN event e ON s.e_id = e.e_id JOIN intersect_proj_event pj ON pj.event_id = e.e_id WHERE
s.flag = 'O' AND
s.modality = 'CT' AND
s.slicingdirection = 'a' AND 
s.dicom_header->'ImageType' ?| array['PRIMARY'] AND
s.dicom_header->'ImageType' ?| array['ORIGINAL'] AND
s.dicom_header->'PatientPosition' ?| array['HFS', 'FFS'] AND
(s.dicom_header->'BodyPartExamined' ?| 
array['Chest', 'CHEST', 'CHEST_ABDOMEN','CHEST-ABDOMEN', 'CT thorax', 'HR_THORAX', 'LUNG', 'Thorax', 'THORAX', 'THORAX ABD', 'THORAX_ABDOMEN', 'THORAX/ABDOMEN','THORAX_HR','THORAX ABDOMEN','Thorax u. Mediastinum','Thx-Abdomen']
OR s.dicom_header->'CodeValue' ?| array['Thorax','RATHLUNG','ctth','cttx','ctthab','CT-HR-Lunge','TH-HR-CT','RATHTHAB','ct-thor','CTTHAB','113691'] OR
LOWER(s.dicom_header->>'SeriesDescription') LIKE '%thorax%' OR 
LOWER(s.dicom_header->>'SeriesDescription') LIKE '%lung%' OR
LOWER(s.dicom_header->>'SeriesDescription') LIKE '%thor%') 

-- AND (s.dicom_header->>'ConvolutionKernel' LIKE 'B6%')
-- AND CAST(s.dicom_header->'SliceThickness' AS FLOAT) = 1.0
AND s.volumeshape[3] >= 100)

-- SELECT ss.e_id FROM series_select ss WHERE ss.max_slices = 1
SELECT ss.dicom_header->>'ConvolutionKernel', count(ss.dicom_header->>'ConvolutionKernel') as sc from series_select ss WHERE ss.max_slices = 1 group by ss.dicom_header->>'ConvolutionKernel' order by sc

-- INSERT INTO intersect_proj_event (proj_name, event_id) SELECT 'lung_b3_3mm', ss.e_id FROM series_select ss WHERE ss.max_slices = 1 ON CONFLICT DO NOTHING 

"""


# SELECT ss.e_id, ss.volumeshape[3] FROM series_select ss WHERE ss.max_slices = 1 
# SELECT DISTINCT ss.dicom_header->'SeriesDescription', ss.volumeshape[3] FROM series_select ss WHERE ss.max_slices = 1 
# SELECT ss.e_id, ss.volumeshape[3] FROM series_select ss WHERE ss.max_slices = 1 

# studymax AS (SELECT s.studyuid as studyuid, max(s.volumeshape[3]) OVER(PARTITION BY p.customer ORDER BY p.total DESC) AS rk) MAX(s.volumeshape[3]) as maxshape from series_select s GROUP BY s.studyuid),

# SELECT s.studyuid, count(*) from series_select s JOIN studymax ON s.sutdyuid = studymax.studyuid

# WHERE s.volumeshape[3] = studymax.maxshape

# GROUP BY s.studyuid 
# """
sql_res = cd.query_to_pandas(SQL_KERNELS)
# sql_res = cur.execute(SQL_KERNELS)
print(sql_res)
# conn.commit()

          ?column?    sc
0   ["Br64f", "3"]     1
1             FC07     1
2             FC10     1
3    ["Q40f", "2"]     1
4                A     1
..             ...   ...
91               B   542
92            B20f   545
93            B31f  1072
94              YB  1426
95            B60f  5851

[96 rows x 2 columns]


In [122]:
pd.set_option('display.max_rows', 100)
sql_res

,?column?,sc
0,"[""Br64f"", ""3""]",1
1,FC07,1
2,FC10,1
3,"[""Q40f"", ""2""]",1
4,A,1
5,B90s,1
6,YA,1
7,CB,1
8,"[""I26f"", ""2""]",1
9,"[""I40f"", ""1""]",1


In [82]:
len(cd.get_events_for_proj('lung_b6_1mm'))

4282

In [ ]:
# -- INSERT INTO intersect_proj_event (proj_name, event_id) SELECT 'allthorax_1mm', ss.e_id FROM series_select ss WHERE ss.max_slices = 1 ON CONFLICT DO NOTHING 


In [ ]:
#      s.dicom_header->>'ConvolutionKernel' LIKE 'B6%' OR
#      s.dicom_header->>'ConvolutionKernel' LIKE 'B7%' OR
#      s.dicom_header->>'ConvolutionKernel' LIKE 'B8%' OR
#      s.dicom_header->>'ConvolutionKernel' LIKE 'B9%' OR
#      s.dicom_header->>'ConvolutionKernel' LIKE '%I5%' OR
#      s.dicom_header->>'ConvolutionKernel' LIKE '%I6%' OR
#      s.dicom_header->>'ConvolutionKernel' LIKE '%I7%' OR
#      s.dicom_header->>'ConvolutionKernel' LIKE '%I8%' OR
#      s.dicom_header->>'ConvolutionKernel' LIKE '%I9%' OR
#      s.dicom_header->>'ConvolutionKernel' LIKE '%I9%' OR 
#      s.dicom_header->>'ConvolutionKernel' LIKE 'FC5%' OR 
#      s.dicom_header->>'ConvolutionKernel' LIKE 'YB'